In [1]:
import os
import datetime
import logging
import sys
import time
import getpass
import pandas 
import requests

from textwrap import dedent

import toloka.client as toloka
import toloka.client.project.template_builder as tb

# False - if problems with tasks in toloka-kit still occure
USE_KIT_FOR_TASKS = True 

PASSWORD = os.environ.get(
    "TOLOKA_PWD", 
    getpass.getpass('Enter your OAuth token: ')
)

# PRODUCTION or SANDBOX, SANDBOX is default
ENVIRONMENT = os.environ.get("TOLOKA_ENV", "SANDBOX")

N_TASKS = os.environ.get("N_TASKS", 2)

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.DEBUG,
    stream=sys.stdout,
)


/home/fatuus/deepschool-cvr-toloka/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Enter your OAuth token:  ········


In [2]:
toloka_client = toloka.TolokaClient(
    PASSWORD, ENVIRONMENT
)
print(toloka_client.get_requester())

[DEBUG] httpx: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] httpx: load_verify_locations cafile='/home/fatuus/deepschool-cvr-toloka/venv/lib/python3.10/site-packages/certifi/cacert.pem'
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d50a84c0>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7fba0c3c9340> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d50a8490>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_

### Preparing Web UI

In [4]:
project = toloka.Project(
    public_name='Write down the digits in an image (toloka-kit)',
    public_description='Look at the image and write down the digits shown on the water meter.',
)

In [6]:
header_viewer = tb.MarkdownViewV1("""1. Look at the image
2. Find boxes with the numbers
3. Write down the digits in black section. (Put '0' if there are no digits there)
4. Put '.'
5. Write down the digits in red section""")

image_viewer = tb.ImageViewV1(tb.InputData('image_url'), rotatable=True)

output_field = tb.TextFieldV1(
    tb.OutputData('value'),
    label='Write down the digits. Format: 365.235',
    placeholder='Enter value',
    hint="Make sure your format of number is '365.235' or '0.112'",
    validation=tb.SchemaConditionV1(
        schema={
            'type': 'string',
            'pattern': r'^\d+\.?\d{0,3}$',
            'minLength': 1,
            'maxLength': 9,
        }
    )
)

task_width_plugin = tb.TolokaPluginV1('scroll', task_width=600)

project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1([header_viewer, image_viewer, output_field]),
    plugins=[task_width_plugin],
)


### Creating Project and Data Specifications

In [7]:
input_specification = {'image_url': toloka.project.UrlSpec()}
output_specification = {'value': toloka.project.StringSpec()}

project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

In [8]:
project.public_instructions = """This task is to solve machine learning problem of digit recognition on the image.<br>
The more precise you read the information from the image the more precise would be algorithm<br>
Your contribution here is to get exact information even if there are any complicated and uncertain cases.<br>
We hope for your skills to solve one of the important science problem.<br><br>
<b>Basic steps:</b><br>
<ul><li>Look at the image and find meter with the numbers in the boxes</li>
<li>Find black numbers/section and red numbers/section</li>
<li>Put black and red numbers separated with '.' to text field</li></ul>"""

In [9]:
project = toloka_client.create_project(project)


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d4dac490>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7fba0c3c9340> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d4c41810>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

In [10]:
!curl https://s3.mds.yandex.net/tlk/dataset/TlkWaterMeters/data.tsv --output data.tsv

raw_dataset = pandas.read_csv('data.tsv', sep='\t', dtype={'value': 'str'})
raw_dataset = raw_dataset[['image_url', 'value']]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0^C


FileNotFoundError: [Errno 2] No such file or directory: 'data.tsv'

In [13]:
pool = toloka.Pool(
    project_id=project.id,
    # Give the pool any convenient name. You are the only one who will see it.
    private_name='Write down the digits in an image.111',
    may_contain_adult_content=False,
    # Set the price per task page.
    reward_per_assignment=0.02,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    # Overlap. This is the number of users who will complete the same task.
    defaults=toloka.Pool.Defaults(default_overlap_for_new_task_suites=3),
    # Time allowed for completing a task page
    assignment_max_duration_seconds=600,
)

In [14]:
pool = toloka_client.create_pool(pool)


[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d4cbdb70>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7fba0c3c9340> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d48651e0>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

In [17]:
urls = [
    "https://2.downloader.disk.yandex.ru/preview/4d586b37bdaa2c56feac0df50988ace5692b7d8171de02ef11746e7c7027eb61/inf/HXXK3JGF4-pHH7QlKGowRbiNAs53ek8hEPIjru4DeDDzU30X0XYLZQ6HrBCzGau1BtHXYs1LiBsYy3pBf0vuaQ%3D%3D?uid=135551846&filename=barcode_03.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=135551846&tknv=v2&size=2066x1368",
    "https://2.downloader.disk.yandex.ru/preview/16ef8969eeb9d96fb2260a58736a7e03bd1f5e4ff663c3a40ed3e3c16dc943c9/inf/RU0Sq2RN6Cb5MUd01dMhZLiNAs53ek8hEPIjru4DeDAzNByppXSizKVS6H2CCfRSyJeW_0q9BaI4bD6PVOd88g%3D%3D?uid=135551846&filename=barcode_04.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=135551846&tknv=v2&size=2066x1368"
]

if USE_KIT_FOR_TASKS:
    tasks = [
        toloka.Task(
            input_values={
                "image_url": url
            }, 
            pool_id=pool.id
        ) for url in urls
    ]
    toloka_client.create_tasks(tasks, allow_defaults=True)
    print(f'Populated pool with {len(tasks)} tasks')
    print(f'To view this pool, go to https://sandbox.toloka.yandex.com/requester/project/{new_project.id}/pool/{new_pool.id}')
else:
    if ENVIRONMENT == "PRODUCTION":
        url = "https://toloka.dev/api/v1/tasks"
    else:
        url = "https://sandbox.toloka.dev/api/v1/tasks"
    headers = {
      'Authorization': f"OAuth {PASSWORD}",
      'Content-Type': 'application/json'
    }
    for url in urls:
        payload = {
          "input_values": {
            "image_url": url
          },
          "pool_id": f"{new_pool.id}"
        }
        response = requests.post(url, headers=headers, json=payload)
        print(response.text)

[DEBUG] httpcore.connection: close.started
[DEBUG] httpcore.connection: close.complete
[DEBUG] httpcore.connection: connect_tcp.started host='sandbox.toloka.dev' port=443 local_address=None timeout=10.0 socket_options=None
[DEBUG] httpcore.connection: connect_tcp.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d46bfb80>
[DEBUG] httpcore.connection: start_tls.started ssl_context=<ssl.SSLContext object at 0x7fba0c3c9340> server_hostname='sandbox.toloka.dev' timeout=10.0
[DEBUG] httpcore.connection: start_tls.complete return_value=<httpcore.backends.sync.SyncStream object at 0x7fb9d4deccd0>
[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'POST']>
[DEBUG] httpcore.http11: r

  0%|                                                   | 0/100 [00:00<?, ?it/s]

[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>


  0%|                                                   | 0/100 [00:00<?, ?it/s]

[DEBUG] httpcore.http11: send_request_headers.complete


...

[DEBUG] httpcore.http11: response_closed.complete


100%|█████████████████████████████████████████| 100/100 [00:15<00:00,  6.32it/s]

[DEBUG] httpcore.http11: send_request_headers.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_headers.complete
[DEBUG] httpcore.http11: send_request_body.started request=<Request [b'GET']>
[DEBUG] httpcore.http11: send_request_body.complete
[DEBUG] httpcore.http11: receive_response_headers.started request=<Request [b'GET']>


[DEBUG] httpcore.http11: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 31 May 2023 19:17:15 GMT'), (b'Content-Type', b'application/json;charset=utf-8'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'x-tlk-request-id', b'a055b2efe003e1a3fc39c8ee9297ebc9'), (b'x-api-ver', b'16.release-278.29044.70d6c91'), (b'vary', b'Origin'), (b'vary', b'Access-Control-Request-Method'), (b'vary', b'Access-Control-Request-Headers'), (b'strict-transport-security', b'max-age=31536000'), (b'x-content-type-options', b'nosniff'), (b'x-xss-protection', b'1; mode=block'), (b'cache-control', b'no-cache, no-store, must-revalidate, max-age=0'), (b'x-frame-options', b'DENY'), (b'x-content-type-options', b'nosniff'), (b'x-qe-streaming', b'true'), (b'vary', b'Accept-Encoding, User-Agent'), (b'content-encoding', b'gzip')])
[INFO] httpx: HTTP Request: GET https://sandbox.toloka.dev/api/v1/operations/c33060f1-63e3-4f5e-96c5-fdac65f790aa/log "HTTP/1.1 

NameError: name 'new_project' is not defined